In [1]:
from osgeo import gdal
import ogr
import matplotlib.pyplot as plt
import numpy as np
import fiona
import xgboost
import pandas as pd

# Set seed
np.random.seed(100)

# Read files 
trainX = np.load('/scratch/slums/bl-slums/gt/final-px-tr-2-Xa')
trainY = np.load('/scratch/slums/bl-slums/gt/final-px-tr-2-Ya')
testX = np.load('/scratch/slums/bl-slums/gt/final-px-te-2-Xa')
testY = np.load('/scratch/slums/bl-slums/gt/final-px-te-2-Ya')

trainY = trainY.ravel()
testY = testY.ravel()
# Reshape training and test data to be suitable to sklearn methods
#print trainX.shape, trainY.shape, testX.shape, testY.shape
#trainX = trainX.reshape(trainX.shape[0], trainX.shape[2])
#valX = valX.reshape(valX.shape[0], valX.shape[2])
#testX = testX.reshape(testX.shape[0], testX.shape[2])
print trainX.shape, trainY.shape, testX.shape, testY.shape

'''
nTestSlums = np.sum(testY==0)

# print nTestSlums
# Pick all other attributes in number equal to nTestSlums randomly
class0Indices = np.where(testY==0)
class1Indices = np.where(testY==1)
class2Indices = np.where(testY==2)

print len(class1Indices[0])
indices1 = class1Indices[0][np.random.choice(len(class1Indices[0]), nTestSlums)]
indices2 = class2Indices[0][np.random.choice(len(class2Indices[0]), nTestSlums)]

class0X = testX[class0Indices[0],:]
class1X = testX[indices1,:]
class2X = testX[indices2,:]

print class0X.shape, class1X.shape, class2X.shape
class0Y = testY[class0Indices]
class1Y = testY[indices1]
class2Y = testY[indices2]

testX = np.vstack((class0X, class1X, class2X))
testY = np.hstack((class0Y, class1Y, class2Y))

print testX.shape, testY.shape
'''

(3547, 18) (3547,) (779, 18) (779,)


/afs/unity.ncsu.edu/users/k/kgadira/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


'\nnTestSlums = np.sum(testY==0)\n\n# print nTestSlums\n# Pick all other attributes in number equal to nTestSlums randomly\nclass0Indices = np.where(testY==0)\nclass1Indices = np.where(testY==1)\nclass2Indices = np.where(testY==2)\n\nprint len(class1Indices[0])\nindices1 = class1Indices[0][np.random.choice(len(class1Indices[0]), nTestSlums)]\nindices2 = class2Indices[0][np.random.choice(len(class2Indices[0]), nTestSlums)]\n\nclass0X = testX[class0Indices[0],:]\nclass1X = testX[indices1,:]\nclass2X = testX[indices2,:]\n\nprint class0X.shape, class1X.shape, class2X.shape\nclass0Y = testY[class0Indices]\nclass1Y = testY[indices1]\nclass2Y = testY[indices2]\n\ntestX = np.vstack((class0X, class1X, class2X))\ntestY = np.hstack((class0Y, class1Y, class2Y))\n\nprint testX.shape, testY.shape\n'

## XGBoost Classifier

# Run the next cell only if you want to use a subset of the 17 features generated:
8 Pansharpened (0-7)
8 Haralick (8-15)
1 NDBI (16)
1 Edge Density (17)


In [2]:


trainX = np.nan_to_num(trainX)
testX = np.nan_to_num(testX)

# trainX = trainX[:,0:7]
# testX = testX[:,0:7]



In [2]:
print np.sum(trainY == 0)
print np.sum(trainY == 1)
print trainX[1:100,17]

2256
1291
[ 34.98723957   0.          47.76380179  56.55156252  53.67838531
  32.02109385  42.76119783  53.41276031  38.26328119  50.07916684
  40.857552    46.73229165  56.04244773  49.41953097  41.73411454
  40.22890621  52.04921881  38.29427094  37.10781246  45.15182294
  26.53151041  46.84296852  10.54088532  22.57812468  33.74322912
   4.50677086  26.4473958    0.           0.           0.          47.69739585
  45.81588544  28.38203123  34.66848956  48.87057273  38.42708323
  47.19713529  36.00989607  38.3473959   33.52187501  48.67578136
  40.05624983  51.38958348  51.63749992  46.79869779  38.56875006
  45.33776029  36.50572936  50.67682298  39.05572923  39.10000017
  47.94088544  42.10598988  46.71015644  42.41145823  31.49869802
  57.80885442  49.02994802  47.67968718  39.62239615  54.39557298
  48.48541662  27.10260434   5.6666667    6.43255209  34.51354165
  31.39687475   0.          38.59973929  19.51015637  27.9083333
  19.29322918   0.           0.           0.          

In [6]:
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from matplotlib import pyplot
model = xgboost.XGBClassifier(nthread=16 ,objective='binary:logistic')
learning_rate = [0.2, 0.3, 0.5, 0.7, 0.9]
n_estimators = [1000,5000,6000, 7000, 10000]
param_grid = dict(learning_rate=learning_rate, n_estimators = n_estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(trainX, trainY)
report(grid_result.cv_results_)


/usr/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Model with rank: 1
Mean validation score: -0.024 (std: 0.012)
Parameters: {'n_estimators': 1000, 'learning_rate': 0.7}

Model with rank: 2
Mean validation score: -0.024 (std: 0.011)
Parameters: {'n_estimators': 10000, 'learning_rate': 0.7}

Model with rank: 3
Mean validation score: -0.024 (std: 0.011)
Parameters: {'n_estimators': 7000, 'learning_rate': 0.7}



In [5]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pickle
#kfold = StratifiedKFold(n_splits = 10, random_state=7)
xgb = xgboost.XGBClassifier(max_depth=500, n_estimators=1000, nthread=8 , objective='binary:logistic', learning_rate = 0.7 )
#results = cross_val_score(xgb, trainX, trainY, cv=kfold)
#print(results)
xgb.fit(trainX,trainY)
result = xgb.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Building','Other'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)

fname = 'VHR-xgboost-2-Cl-model.sav'
pickle.dump(xgb, open(fname, 'wb'))

Overall accuracy = 0.991014120668

Confusion Matrix 
 [[436   1]
 [  6 336]]

Classification Report 
              precision    recall  f1-score   support

   Building       0.99      1.00      0.99       437
      Other       1.00      0.98      0.99       342

avg / total       0.99      0.99      0.99       779




In [6]:
from sklearn.ensemble import RandomForestClassifier

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


#kfold = KFold(n_splits = 10, random_state=100)
rf = RandomForestClassifier(n_estimators = 500)
#results = cross_val_score(rf,trainX, trainY, cv=kfold)
#print results.mean(), results.std()
param_grid = {"max_depth": [3, None],
              "min_samples_split": [2, 3],
              "min_samples_leaf": [1, 3],
              "n_estimators": [500, 1000],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}
grid_search = GridSearchCV(rf, param_grid = param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(trainX, trainY)
report(grid_result.cv_results_)



Model with rank: 1
Mean validation score: -0.025 (std: 0.005)
Parameters: {'bootstrap': False, 'min_samples_leaf': 1, 'n_estimators': 500, 'criterion': 'entropy', 'min_samples_split': 2, 'max_depth': None}

Model with rank: 2
Mean validation score: -0.025 (std: 0.005)
Parameters: {'bootstrap': False, 'min_samples_leaf': 1, 'n_estimators': 500, 'criterion': 'entropy', 'min_samples_split': 3, 'max_depth': None}

Model with rank: 3
Mean validation score: -0.026 (std: 0.005)
Parameters: {'bootstrap': False, 'min_samples_leaf': 1, 'n_estimators': 1000, 'criterion': 'entropy', 'min_samples_split': 3, 'max_depth': None}



In [7]:
rf = RandomForestClassifier(n_estimators = 500, bootstrap = False, min_samples_leaf = 1, min_samples_split = 2, max_depth = None)
rf.fit(trainX,trainY)
result = rf.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Building','Other'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)
fname = 'VHR-rf-2-Cl-model.sav'
pickle.dump(rf, open(fname, 'wb'))


Overall accuracy = 0.98459563543

Confusion Matrix 
 [[435   2]
 [ 10 332]]

Classification Report 
              precision    recall  f1-score   support

   Building       0.98      1.00      0.99       437
      Other       0.99      0.97      0.98       342

avg / total       0.98      0.98      0.98       779




In [8]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(trainX,trainY)
result = nb.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Building','Other'])
print 'Overall accuracy = {}\n'.format(acc)
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)

fname = 'VHR-gnb-2-Cl-model.sav'
pickle.dump(nb, open(fname, 'wb'))

Overall accuracy = 0.962772785623

Confusion Matrix 
 [[430   7]
 [ 22 320]]

Classification Report 
              precision    recall  f1-score   support

   Building       0.95      0.98      0.97       437
      Other       0.98      0.94      0.96       342

avg / total       0.96      0.96      0.96       779




In [9]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=100)
dt.fit(trainX,trainY)
result = dt.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Building','Other'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)

fname = 'VHR-dt-2-Cl-model.sav'
pickle.dump(dt, open(fname, 'wb'))

Overall accuracy = 0.982028241335

Confusion Matrix 
 [[434   3]
 [ 11 331]]

Classification Report 
              precision    recall  f1-score   support

   Building       0.98      0.99      0.98       437
      Other       0.99      0.97      0.98       342

avg / total       0.98      0.98      0.98       779




In [13]:
from sklearn.neighbors import KNeighborsClassifier
cv = StratifiedKFold(n_splits = 10, random_state =7 )
knn = KNeighborsClassifier()
n_neighbors = list(np.arange(3,11,1))
#print n_neighs
params_grid = dict(n_neighbors= n_neighbors)
knn_grid_search = GridSearchCV(estimator = knn, n_jobs = -1, param_grid = params_grid)
knn_grid_result = knn_grid_search.fit(trainX, trainY)
report(knn_grid_result.cv_results_)

knn = KNeighborsClassifier(n_neighbors = 4)
knn.fit(trainX, trainY)
result= knn.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Building','Other'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)

fname = 'VHR-knn-2-Cl-model.sav'
pickle.dump(knn, open(fname, 'wb'))


Model with rank: 1
Mean validation score: 0.920 (std: 0.092)
Parameters: {'n_neighbors': 4}

Model with rank: 2
Mean validation score: 0.919 (std: 0.090)
Parameters: {'n_neighbors': 9}

Model with rank: 3
Mean validation score: 0.919 (std: 0.089)
Parameters: {'n_neighbors': 5}

Overall accuracy = 0.970474967908

Confusion Matrix 
 [[425  12]
 [ 11 331]]

Classification Report 
              precision    recall  f1-score   support

   Building       0.97      0.97      0.97       437
      Other       0.97      0.97      0.97       342

avg / total       0.97      0.97      0.97       779




In [ ]:
'''
from sklearn.svm import SVC

svmclf = SVC(gamma=2,C=0.5)
svmclf.fit(trainX, trainY)
result = svmclf.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Informal','Formal','BG'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)
'''

In [14]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier()
params_grid={
'learning_rate': ["constant", "invscaling", "adaptive"],
'hidden_layer_sizes': [(100,100,100,100), (100,100,100,100,100), (100,100,100,100,100,100,100,100)],
'alpha': [0.0001, 0.00001, 0.01],
'activation': ["logistic", "relu", "tanh"]
}

mlp_grid_search = GridSearchCV(estimator=mlp,param_grid=params_grid,n_jobs=-1,cv=kfold)
mlp_grid_result = mlp_grid_search.fit(trainX, trainY)
report(mlp_grid_result.cv_results_)


Model with rank: 1
Mean validation score: 0.986 (std: 0.005)
Parameters: {'alpha': 1e-05, 'activation': 'logistic', 'learning_rate': 'constant', 'hidden_layer_sizes': (100, 100, 100, 100)}

Model with rank: 2
Mean validation score: 0.984 (std: 0.007)
Parameters: {'alpha': 1e-05, 'activation': 'relu', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (100, 100, 100, 100)}

Model with rank: 3
Mean validation score: 0.984 (std: 0.007)
Parameters: {'alpha': 1e-05, 'activation': 'logistic', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (100, 100, 100, 100, 100)}

Model with rank: 3
Mean validation score: 0.984 (std: 0.008)
Parameters: {'alpha': 0.0001, 'activation': 'relu', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (100, 100, 100, 100, 100, 100, 100, 100)}



In [15]:
mlp = MLPClassifier(activation='logistic', learning_rate = 'constant',hidden_layer_sizes=(100, 100, 100, 100), alpha = 0.00001)
mlp.fit(trainX, trainY)
result = mlp.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Building','Other'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)

fname = 'VHR-mlp-2-Cl-model.sav'
pickle.dump(mlp, open(fname, 'wb'))

Overall accuracy = 0.978177150193

Confusion Matrix 
 [[432   5]
 [ 12 330]]

Classification Report 
              precision    recall  f1-score   support

   Building       0.97      0.99      0.98       437
      Other       0.99      0.96      0.97       342

avg / total       0.98      0.98      0.98       779




In [ ]:
'''
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

gp = GaussianProcessClassifier(1.0 * RBF(1.0))
gp.fit(trainX, trainY)
result = gp.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Slum','Urban','Other'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)
'''

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
adb = AdaBoostClassifier()
params_grid = dict(n_estimators=[50,100,500,1000,5000], learning_rate=[0.01, 0.007, 0.0001, 0.1, 0.0007])
adb_grid_search = GridSearchCV(estimator=adb, param_grid = params_grid, cv=kfold)
adb_search_result = adb_grid_search.fit(trainX, trainY)


NameError: name 'report' is not defined

In [7]:
report(adb_search_result.cv_results_)

Model with rank: 1
Mean validation score: 0.994 (std: 0.003)
Parameters: {'n_estimators': 500, 'learning_rate': 0.1}

Model with rank: 1
Mean validation score: 0.994 (std: 0.003)
Parameters: {'n_estimators': 1000, 'learning_rate': 0.1}

Model with rank: 3
Mean validation score: 0.993 (std: 0.003)
Parameters: {'n_estimators': 5000, 'learning_rate': 0.01}



In [10]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pickle
adb = AdaBoostClassifier(n_estimators=500, learning_rate=0.1)
adb.fit(trainX, trainY)
result = adb.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Building','Other'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)

fname = 'VHR-adaboost-2-Cl-model.sav'
pickle.dump(adb, open(fname, 'wb'))

Overall accuracy = 0.992297817715

Confusion Matrix 
 [[436   1]
 [  5 337]]

Classification Report 
              precision    recall  f1-score   support

   Building       0.99      1.00      0.99       437
      Other       1.00      0.99      0.99       342

avg / total       0.99      0.99      0.99       779




In [ ]:
clfs = [xgb, rf, nb, dt, knn, mlp, adb]
clf-names = ['xgb', 'rf', 'nb', 'dt', 'knn', 'mlp', 'adb']

In [ ]:
'''
from osgeo import gdal
import ogr
import matplotlib.pyplot as plt
import numpy as np
import fiona

driver = gdal.GetDriverByName('GTiff')
rasterFileName1 = "/scratch/slums/bl-slums/raw-img/PS_mosaic_415.tif"
dataset1 = gdal.Open(rasterFileName1)
rasterFileName2 = "/scratch/slums/bl-slums/raw-img/PS_mosaic_415_NDBI.tif"
dataset2 = gdal.Open(rasterFileName2)
rasterFileName3 = "/scratch/slums/bl-slums/features/pan/haralick/PAN_mosaic_415-simple-50.tif"
dataset3 = gdal.Open(rasterFileName3)
rasterFileName4 = "/scratch/slums/bl-slums/raw-img/PAN_mosaic_415_edgeDensity.tif"
dataset4 = gdal.Open(rasterFileName4)


imggeotrans = dataset1.GetGeoTransform()


bands1=[]
bands2=[]
bands3=[]
bands4=[]
data_all_bands = []
cols = dataset1.RasterXSize
rows = dataset1.RasterYSize

transform = dataset1.GetGeoTransform()
xOrigin = transform[0]
yOrigin = transform[3]
pixelWidth = transform[1]
pixelHeight = -transform[5]

print xOrigin, yOrigin, pixelWidth, pixelHeight

for i in range(8):
    bands1.append(dataset1.GetRasterBand(i+1)) 
    # data_all_bands.append(band.ReadAsArray(0,0,cols,rows).astype(np.float))
bands2.append(dataset2.GetRasterBand(1)) 
for i in range(8):
    bands3.append(dataset3.GetRasterBand(i+1)) 
bands4.append(dataset4.GetRasterBand(1)) 

#all_data = np.zeros((rows, cols, 18))
#for k in range(8):
#    all_data[:,:,k] = bands1[k].ReadAsArray(0,0,cols, rows)
    
#for k in range(8):
#    all_data[:,:,k+8] = bands3[k].ReadAsArray(0,0,cols, rows)
#all_data[:,:,16] = bands2[0].ReadAsArray(0,0,cols, rows)
#all_data[:,:,17] = bands4[0].ReadAsArray(0,0,cols, rows)
'''

In [ ]:
'''
import time
start_t = time.time()
result_raster = np.zeros((rows,cols))-1
noneData = False
for i in range(rows):
    curr_row = all_data[i,:,:]
    result = rf.predict(curr_row)    
    result_raster[i,:] = result
    if i%10 ==0:
        print '{} rows completed'.format(i)
        
end_t = time.time()

print 'Total exec time = {}'.format(end_t-start_t)
'''

In [ ]:
'''
import time
start_t = time.time()
result_raster = np.zeros((rows,cols), dtype=np.int)-1
noneData = False
for i in range(rows):
    if i%100==0:
        print 'Finished {} pixels'.format(i)
    curr_row = np.zeros((cols,18))-1
    for k in range(8):
        data = bands1[k].ReadAsArray(0,i,cols,1)
        data =data.astype(np.float)
        print data.shape
        curr_row[:,k] = data 
    for k in range(8):
        data = bands3[k].ReadAsArray(0,i,cols,1)
        print data.shape
        data = data.astype(np.float)
        curr_row[:,k+8] = data 
    data = bands2[0].ReadAsArray(0,i,cols,1)
    curr_row[:,16] = data
    data = bands4[0].ReadAsArray(0,i,cols,1)
    curr_row[:,17] = data
    #print curr_row.shape
    #result = rf.predict(curr_row)    
    #result_raster[i,:] = result
        
end_t = time.time()

print 'Total exec time = {}'.format(end_t-start_t)
'''        

In [ ]:
'''
from utils import io

io.write_image(cols, rows, 1,imggeotrans, result_raster, '/scratch/slums/bl-slums/clf-img/clf-xgboost-building.tif')
'''